# Analysis of selected isomers of (H$_2$O)$_5$ water clusters

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import pprint
%matplotlib inline 

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit import rdBase
from rdkit.Chem import rdmolfiles
from rdkit.Chem import rdDetermineBonds
print(rdBase.rdkitVersion)
import os,time
print( time.asctime())
import py3Dmol
import xyz2mol as x2m

from ipywidgets import interact,fixed,IntSlider
import ipywidgets

2023.09.6
Tue Jul 16 12:23:22 2024


In [3]:
# setup
gridspacing=0.05
modes_dir=Path(os.getcwd(), 'W5_struc_4', 'results_gridspacing'+str(gridspacing)).resolve()

# handy functions
def prepare_view(moldict, p=None):
    if p is None:
        p = py3Dmol.view(width=400,height=400)
    p.removeAllModels()
    for key, mol in moldict.items():
        mb = Chem.MolToMolBlock(mol)
        p.addModel(mb,'sdf')
    p.setStyle({'stick':{'radius':'0.1'}})
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    return p   

def prepare_view_single(onemol, p=None):
    if p is None:
        p = py3Dmol.view(width=400,height=400)
    p.removeAllModels()
    mb = Chem.MolToMolBlock(onemol)
    p.addModel(mb,'sdf')
    p.setStyle({'stick':{'radius':'0.1'}})
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    return p  


In [4]:
ftraj_all={}
for m in modes_dir.glob('*'):
    mode=m.stem.split('mode')[1]
    chem_suppl={}
    for geom in modes_dir.glob('*/coordinates/*'):
        g=geom.stem
        if 'geom_0' in g:
            Ns=0
        elif 'minus' in g:
            Ns=-int(g.split('_')[2])
        elif 'plus' in g:
            Ns=int(g.split('_')[2])
        raw_mol = rdmolfiles.MolFromXYZFile(str(geom))
        mol = Chem.Mol(raw_mol)
        rdDetermineBonds.DetermineConnectivity(mol)
        chem_suppl[Ns]=mol
    ftraj_all[mode]=chem_suppl

IndexError: list index out of range

# Description

In this example, we consider one selected isomer of "W5" water clusters with 5 water molecules; we refer to it as "W5_struc_4".
We show its equilibrium (optimized) geometry below. We refer to the energy of this equilibrium geometry as "E[0]".

The water cluster consists of 5 water molecules; thus, there are 39 normal modes of harmonic vibrations (according to the formula: Nmodes=3*Natoms-6, where "Natoms" is the number of all atoms in the system).

Each normal mode represents the movement of all atoms that does not rotate or translate the molecule in space.
Then, we consider molecular structures "distorted" along each mode.

In other words:
* every atom "At" in a system has an equilibrium position, At[0]=(x[0], y[0], z[0])
* one selected normal mode ("Nmode") is characterized by a certain frequency ("freq_mode") and a set of vectors demonstrating the direction of movement of each atom (there are "Natoms" of these vectors; v[at] = (vx[at], vy[at], vz[at]));
* in the generated "distorted" structures, the position of each atom is calculated as (pseudocode):

  ```
  # prepare:
  dx, dy, dz  - grid spacing in x/y/z directions
  epsilon=1.5 - this is an arbitrarily chosen value (to ensure the step is slightly larger than dx/dy/dz)
  Nsteps=9    - this is an arbitrarily chosen number of steps
  
  for n in -Nsteps, ..., 0, ..., Nsteps:
    ex=epsilon*n*dx
    ey=epsilon*n*dy
    ez=epsilon*n*dz
    At[n] =(x[0] + ex*vx[At], y[0] + ey*vy[At], z[0] + ez*vz[At])
  ```

* then, for each of these geometries, we calculate the ED (electron density) and BNP (bare nuclear potential) functions (on grids created for each of these geometries).

Therefore, in total, we have "2*Nsteps+1" structures (with structure "0" corresponding to the equilibrium geometry) for 39 normal modes of "W5_struc_4" cluster, which in total generates 741 datasets.

### Example 1: one selected normal mode

In [ ]:
# Choose the normal mode to demonstrate; in this example - from 0 to 38;
# then, rerun the cells below
selected=20

In [ ]:
print("This shows the equilibrium structure of W5_struc_4")
mol=ftraj_all["0"][0]
viewer=prepare_view_single(mol)
viewer.show()

In [ ]:
# we now track the nuclear displacement along the selected normal mode:

# equilibrium geometry:
f_0 = Path(modes_dir, "molecule_optimized.xyz")
N_atoms=int(open(f_0,"r").readline().rstrip())
atoms_df = pd.read_csv(f_0, skiprows=2,sep='\s+',names=['at','x0','y0','z0'],skip_blank_lines=True)
#pprint.pprint(atoms_df)

# tracking along the mode:
f_modes = Path(modes_dir, "modes.txt")
with open(f_modes, "r") as f:
    lines=f.readlines()
    selected_mode=lines[selected-1].strip().split()
    qx=[]
    qy=[]
    qz=[]
    for i in range(N_atoms):
        qx.append(float(selected_mode[0]))
        qy.append(float(selected_mode[1]))
        qz.append(float(selected_mode[2]))
        del selected_mode[0:3]
atoms_df["qx_"+str(selected)] = pd.Series(qx)
atoms_df["qy_"+str(selected)] = pd.Series(qy)
atoms_df["qz_"+str(selected)] = pd.Series(qz)
pprint.pprint(atoms_df)
#for i in range(N_atoms):
#    print(at[i])        
        #for x in selected_mode:
    #xyz_displ=[x[0:3] for x in selected

In [ ]:

def prepare_view_single_with_arrows(onemol, onemol_df, p=None):
    if p is None:
        p = py3Dmol.view(width=400,height=400)
    p.removeAllModels()
    mb = Chem.MolToMolBlock(onemol)
    p.addModel(mb,'sdf')
    p.setStyle({'stick':{'radius':'0.1'}})
    p.setBackgroundColor('0xeeeeee')
    for ind in onemol_df.index:
        xs=onemol_df["qx_"+str(selected)][ind]
        ys=onemol_df["qy_"+str(selected)][ind]
        zs=onemol_df["qz_"+str(selected)][ind]
        xe=xs+1
        ye=ys+1
        ze=zs+1
        p.addArrow({
                  start: {x:xs, y:ys, z:zs},
                  end: {x:xe, y:ye, z:ze},
                  radius: 1.0,
                  radiusRadio:1.0,
                  mid:1.0,
                  clickable:true,
                  })
    p.zoomTo()
    return p  



In [ ]:
viewer=prepare_view_single_with_arrows(mol, atoms_df)
viewer.show()

In [ ]:
print("This shows all structures within the normal mode, ", selected)
print("We overlay the structures to show the range of motion of O atoms (red) and H atoms (white).")
print("The number of overlaid structures: ", len(ftraj_all[str(selected)]))
viewer=prepare_view(ftraj_all[str(selected)])
viewer.show()

In [ ]:
print("Use the slider to see how atoms move along the selected normal mode")

geoms=ftraj_all[str(selected)]

def mol_viewer(idx):
    mol = geoms[idx]
    return prepare_view_single(mol).show()

minv=min(geoms)
maxv=max(geoms)-1

interact(mol_viewer, idx=ipywidgets.IntSlider(min=minv,max=maxv, step=1))